In [334]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
from tqdm import tqdm
import ssl
import warnings
warnings.filterwarnings('ignore')

--------------------------------------------------------
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
--------------------------------------------------------

In [335]:
#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]       # 뉴스 제목
link_text=[]        # 뉴스 링크
source_text=[]      # 신문사
date_text=[]        # 날짜
contents_text=[]    # 내용
result={}           # 딕셔너리 형태로 저장

In [336]:
def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
        response = requests.get(url,headers=headers, verify=False)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사

        page += 10
        print(page, title_text)
        
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)

In [337]:
def main():
    info_main = input("="*50+"\n입력 형식에 맞게 입력해주세요."+"\n시작하시려면 Enter를 눌러주세요.\n"+"="*50)
    
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
    query = input("검색어 입력: ")  
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
    e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()

11 ['LG화학, 삼표시멘트·현대로템과 폐플라스틱 자원화 협력', '[더벨][전쟁 특수, 방산 다시보기]현대로템, K2로 폴란드 거쳐 유럽 진격', '대아티아이·푸른기술 하락 이어 현대로템 보합, 남북철도 관련주 지지부진', '방산주 장중 강세, 폴란드 방산수출 기대감에 상승세 이어가', '尹 첫 순방서 안보 협력 논의… 방산업계, 수출 기대감 커져', "[현대차그룹] 등락률 살펴보니…현대오토에버 4.0% 상승 '눈길'", "尹이 직접 세일즈한 'K-방산'…해외 순방 성과 기대감에 방산업체 주가↑", "[상반기 리그테이블-채권주관] '넘사벽' KB證, 10년째 1위 수성", "[현대차그룹] 시가총액 1조1349억원 증가...현대건설 4.16% 상승 '주목'", "[뒷북경제] 용산으로 간 철도노조… '민영화' 없다는데 왜?"]


In [338]:
# 다른데서 가져온 뉴스 내용 크롤링
for i in tqdm(link_text):
    #각 기사 html get하기
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    news = requests.get(i, headers=headers, verify=False)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 본문 가져오기
    content = news_html.findAll('article')
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    contents_text.append(content)

    # try:
    #     html_date = news_html.find('fa fa-clock-o fa-fw')
    #     news_date = html_date.attrs['data-date-time']
    # except AttributeError:
    #     news_date = news_html.find('fa fa-clock-o fa-fw')
    #     news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))

    # # 날짜 가져오기
    # html_date = news_html.findAll("clock")
    # print(html_date)
    # date_text.append(html_date)

100%|██████████| 10/10 [00:02<00:00,  3.76it/s]


In [339]:
print(date_text)

['2022.07.01.', '2022.07.01.', '2022.07.01.', '2022.07.01.', '2022.07.01.', '2022.07.02.', '2022.07.02.', '2022.07.01.', '2022.07.01.', '2022.07.02.']


In [340]:
print(contents_text)

['[\n2023-07-04 17:13 (화) \n\n\n\n전체메뉴\n\n\n\n\n기사검색\n\n\n\n\n\n\n기사검색\n\n\n검색\n\n닫기\n\n\n\n\n\n\n\n상세검색\n\n\n\n\n, \n\n\n로그인\n회원가입\n구독신청\n지면보기\n\n\n\nfacebook\ntwitter\nblog\npost\ncafe-naver\ncafe-daum\n모바일웹\n\n, \n\n\n\n\n\n\t// 기사면 상단 네비게이션 호출\n\tarticleHeader.setScroll();\n\n\t$(window).scroll(function() {\n        var currY = $(this).scrollTop();\n        var postHeight = $(this).height();\n        var scrollHeight = $(\'body\').height();\n        var scrollPercent = (currY / (scrollHeight - postHeight)) * 100;\n        $(\'.aht-bar\').css(\'width\', scrollPercent +"%");\n    });\n\n\n\n\n이전 기사보기\n다음 기사보기\n\n\n\n\nLG화학, 삼표시멘트·현대로템과 폐플라스틱 자원화 협력\n\n\n바로가기\n복사하기\n본문 글씨 줄이기\n본문 글씨 키우기\n\n\n\n\n\n\n스크롤 이동 상태바\n\n\n\n\n\n\nLG화학, 삼표시멘트·현대로템과 폐플라스틱 자원화 협력\n\n\n기업·산업\n입력 2022.07.01 09:46\n수정 2022.07.01 12:51\n\n\n\n기자명\n\t\t\t김정필 \xa0thinkbig@newscj.com\t\t\n\n\n\n페이스북(으)로 기사보내기\n\n\n트위터(으)로 기사보내기\n\n\nURL복사(으)로 기사보내기\n\n\n\n바로가기\n기사스크랩하기\n다른 공유 찾기\n\n\n본문 글씨 줄이기\n가\n본문 글씨 키우기\n\n\n\n\n\

In [341]:
df = pd.DataFrame({"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text })
df

,date,title,source,contents,link
0,2022.07.01.,"LG화학, 삼표시멘트·현대로템과 폐플라스틱 자원화 협력",천지일보,[\n2023-07-04 17:13 (화) \n\n\n\n전체메뉴\n\n\n\n\n...,http://www.newscj.com/article/20220701580097
1,2022.07.01.,"[더벨][전쟁 특수, 방산 다시보기]현대로템, K2로 폴란드 거쳐 유럽 진격",더벨,[],http://www.thebell.co.kr/front/free/contents/n...
2,2022.07.01.,"대아티아이·푸른기술 하락 이어 현대로템 보합, 남북철도 관련주 지지부진",이코노뉴스,[\n\n\n기자명 \n\t\t\t\t\t이준우 기자\t\t\t\t\n 입력 202...,http://www.econonews.co.kr/news/articleView.ht...
3,2022.07.01.,"방산주 장중 강세, 폴란드 방산수출 기대감에 상승세 이어가",비즈니스포스트,[],https://www.businesspost.co.kr/BP?command=arti...
4,2022.07.01.,"尹 첫 순방서 안보 협력 논의… 방산업계, 수출 기대감 커져",뉴데일리,[],https://biz.newdaily.co.kr/site/data/html/2022...
5,2022.07.02.,[현대차그룹] 등락률 살펴보니…현대오토에버 4.0% 상승 '눈길',핀포인트뉴스,[\n\n\n기자명 \n\t\t\t\t\t한강희 기자\t\t\t\t\n 입력 202...,http://www.pinpointnews.co.kr/news/articleView...
6,2022.07.02.,尹이 직접 세일즈한 'K-방산'…해외 순방 성과 기대감에 방산업체 주가↑,머니투데이,[],http://news.mt.co.kr/mtview.php?no=20220701170...
7,2022.07.01.,"[상반기 리그테이블-채권주관] '넘사벽' KB證, 10년째 1위 수성",연합인포맥스,[\n\n\n기자명 \r\n\t\t\t\t\t박준형 기자\t\t\t\t\n 입력 2...,http://news.einfomax.co.kr/news/articleView.ht...
8,2022.07.01.,[현대차그룹] 시가총액 1조1349억원 증가...현대건설 4.16% 상승 '주목',핀포인트뉴스,[\n\n\n기자명 \n\t\t\t\t\t한강희 기자\t\t\t\t\n 입력 202...,http://www.pinpointnews.co.kr/news/articleView...
9,2022.07.02.,[뒷북경제] 용산으로 간 철도노조… '민영화' 없다는데 왜?,서울경제언론사 선정,[],https://www.sedaily.com/NewsView/268D6IEBN5


In [342]:
# 데이터프레임 파일 저장
now = datetime.now() #파일이름 현 시간으로 저장하기
df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}_{}_{}_{}.xlsx'.format(query, now.strftime('%Y%m%d_%H%M%S'), s_date, e_date),encoding='utf-8-sig',index=False)

NameError: name 'query' is not defined